In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [11]:
# Filter the dataset where total_votes count is equal to or greater than 20
reviews_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
filtered_review_df = df.filter(reviews_df.total_votes >= 20)
filtered_review_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36886257|R1T4FVSABO0IDP|B00V03D9KY|     966609438|Lukas Lk-7950 WD ...|      Automotive|          5|            8|         31|   N|                Y|      Great Value!!!|LK-7950 is an out...| 2015-08-31|
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|    

In [13]:
# Create a DataFrame to retrieve all rows where the number of helpful_votes/total_votes is equal to or greater than 50%
helpful_review_df = filtered_review_df.filter((filtered_review_df.helpful_votes/filtered_review_df.total_votes) >= .5)
helpful_review_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [17]:
# Create a DataFrame where review were part of the Vine program
paid_reviews = helpful_review_df.filter(helpful_review_df.vine == "Y")
paid_reviews.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35560012| REKN4PLM5174P|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|          219|        225|   Y|                N|Head 2 Head, It's...|Having come into ...| 2015-07-08|
|         US|   52693410| RQOG8IIJX7CKB|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|    

In [18]:
# Create a DataFrame where review were not part of the Vine program
unpaid_reviews = helpful_review_df.filter(helpful_review_df.vine == "N")
unpaid_reviews.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [21]:
# Total number of reviews
total_reviews = helpful_review_df.count()
total_reviews

24824

In [22]:
# Total number of five stars reviews
five_stars_df = helpful_review_df.filter(helpful_review_df.star_rating == 5)
total_five_stars = five_stars_df.count()
total_five_stars

12840

In [25]:
# Percentage of five stars reviews
pt_five_stars = round((total_five_stars / total_reviews * 100),2)
pt_five_stars

51.72

In [31]:
# Percentage of five stars reviews (paid)
total_paid_reviews = paid_reviews.filter(paid_reviews.star_rating == 5).count()
pt_paid_reviews = round((total_paid_reviews / total_five_stars * 100),2)
pt_paid_reviews

0.26

In [32]:
# Percentage of five stars reviews (unpaid)
total_unpaid_reviews = unpaid_reviews.filter(unpaid_reviews.star_rating == 5).count()
pt_unpaid_reviews = round((total_unpaid_reviews / total_five_stars * 100),2)
pt_unpaid_reviews

99.74